In [1]:
import transformers
import tokenizers
import numpy as np
import datasets

dataset = datasets.load_from_disk('./hf_dataset')
tokenized_datasets = datasets.load_from_disk('./hf_dataset_tok')

In [2]:
id = 0
max_length = 1024
tokens = np.load('../data/formatted/tokens.npy', allow_pickle=True)

if dataset == None:
    train = np.load('../data/shuffled/dataset_train.npy', allow_pickle=True)
    test = np.load('../data/shuffled/dataset_test.npy', allow_pickle=True)
    midi_train = np.load('../data/shuffled/midi_train.npy', allow_pickle=True)
    midi_test = np.load('../data/shuffled/midi_test.npy', allow_pickle=True)

    dataset_dict = {
        'train': datasets.Dataset.from_dict({'text' : train, 'midi' : midi_train}),
        'test': datasets.Dataset.from_dict({'text' : test,'midi' : midi_test}),
        }

    dataset = datasets.DatasetDict(dataset_dict)
    dataset.save_to_disk('../data/hf_dataset')

In [3]:
tokenizer = transformers.PreTrainedTokenizerFast(tokenizer_file='hf_tokenizer')
if tokenizer == None:
    tokenizer = tokenizers.Tokenizer(tokenizers.models.WordLevel(unk_token="[UNK]"))
    trainer = tokenizers.trainers.WordLevelTrainer(special_tokens=["[UNK]","<pad>"])
    tokenizer.train_from_iterator(dataset['train']['text'],trainer=trainer)
    # [x for x in tokens.tolist() if x not in sorted(list(tokenizer.get_vocab().keys()))]
    tokenizer.save('../data/hf_tokenizer')
    
# tokenizer.get_vocab()

In [4]:
# tokenizer.enable_padding(pad_id = tokenizer.token_to_id('<pad>'), pad_token='<pad>', length=max_length)
# print(tokenizer.token_to_id('<pad>'))
tokenizer

PreTrainedTokenizerFast(name_or_path='', vocab_size=195, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	4: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	52: AddedToken("<end>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	53: AddedToken("<start>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [5]:
# test_str = dataset['train'][0]['text'][:10]
# print(test_str)

In [6]:
tokenizer.vocab_size
tokenizer.pad_token = '<pad>'

In [7]:
if tokenized_datasets == None:
    def tokenize(element):
        outputs = tokenizer.convert_tokens_to_ids(
            element["text"],
        )
        return {"input_ids": outputs}


    tokenized_datasets = dataset.map(
        tokenize, remove_columns=dataset["train"].column_names
    )
    tokenized_datasets
    tokenized_datasets.save_to_disk('hf_dataset_tok')

In [8]:
data_collator = transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False, )

In [9]:
# model = transformers.AutoModelForCausalLM.from_config(
#     transformers.GPT2Config(
#         vocab_size=200, #tokenizer.vocab_size,
#         n_positions=max_length,
#         n_embd=128,
#         n_layer=2,
#         n_head=4,
#         # bos_token_id=tokenizer.token_to_id('<start>'),
#         # eos_token_id=tokenizer.token_to_id('<end>'),
#     )
# )


model = transformers.AutoModelForCausalLM.from_config(
    transformers.MambaConfig(
        vocab_size=200,
        hidden_size=128,
        state_size=4,
        num_hidden_layers=8,
        pad_token_id=tokenizer.pad_token_id,
        # bos_token_id=tokenizer.token_to_id('<start>'),
        # eos_token_id=tokenizer.token_to_id('<end>'),
    )
)

The fast path is not available because on of `(selective_state_update, selective_scan_fn, causal_conv1d_fn, causal_conv1d_update, mamba_inner_fn)` is None. Falling back to the naive implementation. To install follow https://github.com/state-spaces/mamba/#installation and https://github.com/Dao-AILab/causal-conv1d


In [10]:
model_size = sum(t.numel() for t in model.parameters())
print(f"GPT-2 size: {model_size/1000**2:.1f}M parameters")

GPT-2 size: 0.9M parameters


In [11]:
args = transformers.TrainingArguments(
    output_dir="workspace/data/hf_gpt2_chords",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    evaluation_strategy="steps",
    eval_steps=1_000,
    logging_steps=100,
    gradient_accumulation_steps=8,
    num_train_epochs=200,
    weight_decay=0.1,
    warmup_steps=1_000,
    lr_scheduler_type="cosine",
    learning_rate=5e-4,
    save_steps=5_000,
    # fp16=True,
    report_to=None,
)

trainer = transformers.Trainer(
    model=model,
    tokenizer=tokenizer,
    args=args,
    data_collator=data_collator,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [12]:
trainer.train()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: dazzid (music_gpt). Use `wandb login --relogin` to force relogin


[1711468170.568244] [d72355bbe943:1003952:f]        vfs_fuse.c:281  UCX  ERROR inotify_add_watch(/tmp) failed: No space left on device


/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss,Validation Loss
